# Performance Analysis Project

Work by: Gonçalo Dias and Vicente Bandeira

## Introduction

This notebook serves as both an implementation and a report for the work proposed to us in the CDLE cadeira in IACD. In this report, we present the findings of our comprehensive analysis of machine learning performance using various Python libraries for big data processing. As the scale and complexity of data continue to grow, the choice of tools and libraries becomes critical in ensuring efficient and effective machine learning workflows. To address this, we executed a full Machine Learning Pipeline using several prominent big data libraries: PySpark, Dask, Modin, JobLib, Rapids and Koalas

## Repeating the NYC taxi driver dataset study

Source: https://www.databricks.com/blog/2021/04/07/benchmark-koalas-pyspark-and-dask.html

In [1]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [2]:
import databricks.koalas as ks
import dask
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster
import pandas as pd
import numpy as np
import time

print('pandas version: %s' % pd.__version__)
print('numpy version: %s' % np.__version__)
print('koalas version: %s' % ks.__version__)
print('dask version: %s' % dask.__version__)

pandas version: 1.1.5
numpy version: 1.19.5
koalas version: 1.7.0
dask version: 2021.04.1


### Dataset information

In [3]:
url = "../yellow_tripdata_2011-01.parquet"
koalas_data = ks.read_parquet(url)

In [4]:
print("Number of rows:", len(koalas_data))
print()
print(koalas_data.head(20))

Number of rows: 13464997

    VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  payment_type  fare_amount  extra  mta_tax  tip_amount  tolls_amount  improvement_surcharge  total_amount  congestion_surcharge  airport_fee
0          2  2011-01-01 00:10:00   2011-01-01 00:12:00                4            0.0           1               None           145           145             1          2.9    0.5      0.5        0.28           0.0                    0.0          4.18                   NaN          NaN
1          2  2011-01-01 00:04:00   2011-01-01 00:13:00                4            0.0           1               None           264           264             1          5.7    0.5      0.5        0.24           0.0                    0.0          6.94                   NaN          NaN
2          2  2011-01-01 00:14:00   2011-01-01 00:16:00                4            0.0           1           

In [5]:
koalas_data.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                   int64
trip_distance                   float64
RatecodeID                        int64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
dtype: object

In [6]:
expr_filter = (koalas_data['tip_amount'] >= 1) & (koalas_data['tip_amount'] <= 5)
 
print(f'Filtered data is {len(koalas_data[expr_filter]) / len(koalas_data) * 100}% of total data')

Filtered data is 35.84216914418919% of total data


### Experiment preparation

In [7]:
def benchmark(f, df, benchmarks, name, **kwargs):
    """Benchmark the given function against the given DataFrame.

    Parameters
    ----------
    f: function to benchmark
    df: data frame
    benchmarks: container for benchmark results
    name: task name


    Returns
    -------
    Duration (in seconds) of the given operation
    """

    start_time = time.time()
    ret = f(df, **kwargs)
    benchmarks['duration'].append(time.time() - start_time)
    benchmarks['task'].append(name)
    print(f"{name} took: {benchmarks['duration'][-1]} seconds")

def get_results(benchmarks):
    """Return a pandas DataFrame containing benchmark results."""
    return pd.DataFrame.from_dict(benchmarks)

In [8]:
paths = ["../yellow_tripdata_201"+str(i)+"-01.parquet" for i in range(1,4)]

## Koalas

In this section we utilized the Koalas library for our experiments, trying it on Standard operations, operations with filtering and operations with filtering and caching.

In [9]:
koalas_data = ks.read_parquet(paths[0])

In [10]:
koalas_benchmarks = {
    'duration': [],     # in seconds
    'task': []
}

#### Standard operations

In [11]:
def read_file_parquet(df=None):
    return ks.read_parquet("../yellow_tripdata_2011-01.parquet")

def count(df=None):
    return len(df)

def count_index_length(df=None):
    return len(df.index)

def mean(df):
    return df.fare_amount.mean()

def standard_deviation(df):
    return df.fare_amount.std()

def mean_of_sum(df):
    return (df.fare_amount + df.tip_amount).mean()

def sum_columns(df):
    x = df.fare_amount + df.tip_amount
    return x

def mean_of_product(df):
    return (df.fare_amount * df.tip_amount).mean()

def product_columns(df):
    x = df.fare_amount * df.tip_amount
    return x

def value_counts(df):
    val_counts = df.fare_amount.value_counts()
    return val_counts


#   In the original experiment, the following two functions used the longitude and latitude values of the pickup and the dropout places.
#   Since the datasets provided by NYC TLC Trip Record Data no longer have longitude and latitude values (only the pickup and dropout places IDs),
# we used arbitrary longitude and latitude values. The goal of this experiment is to compare the computational cost of the calculations, hence
# the values are not relevant.
def complicated_arithmetic_operation(df):
    start_lon,end_lon = np.random.randint(-180,180),np.random.randint(-180,180)
    start_lat,end_lat = np.random.randint(-90,90),np.random.randint(-90,90)
    
    theta_1 = start_lon
    phi_1 = start_lat
    theta_2 = end_lon
    phi_2 = end_lat
    temp = (np.sin((theta_2 - theta_1) / 2 * np.pi / 180) ** 2
           + np.cos(theta_1 * np.pi / 180) * np.cos(theta_2 * np.pi / 180) * np.sin((phi_2 - phi_1) / 2 * np.pi / 180) ** 2)
    ret = np.multiply(np.arctan2(np.sqrt(temp), np.sqrt(1-temp)),2)
    return ret

def mean_of_complicated_arithmetic_operation(df):
    start_lon,end_lon = np.random.randint(-180,180),np.random.randint(-180,180)
    start_lat,end_lat = np.random.randint(-90,90),np.random.randint(-90,90)
    
    theta_1 = start_lon
    phi_1 = start_lat
    theta_2 = end_lon
    phi_2 = end_lat
    temp = (np.sin((theta_2 - theta_1) / 2 * np.pi / 180) ** 2
           + np.cos(theta_1 * np.pi / 180) * np.cos(theta_2 * np.pi / 180) * np.sin((phi_2 - phi_1) / 2 * np.pi / 180) ** 2)
    ret = np.multiply(np.arctan2(np.sqrt(temp), np.sqrt(1-temp)),2) 
    return ret.mean()

def groupby_statistics(df):
    gb = df.groupby(by='passenger_count').agg(
      {
        'fare_amount': ['mean', 'std'], 
        'tip_amount': ['mean', 'std']
      }
    )
    return gb

other = ks.DataFrame(groupby_statistics(koalas_data).to_pandas())
other.columns = pd.Index([e[0]+'_' + e[1] for e in other.columns.tolist()])

def join_count(df, other):
    return len(df.merge(other.spark.hint("broadcast"), left_index=True, right_index=True))

def join_data(df, other):
    ret = df.merge(other.spark.hint("broadcast"), left_index=True, right_index=True)
    return ret

In [12]:
benchmark(read_file_parquet, df=None, benchmarks=koalas_benchmarks, name='read file')
benchmark(count, df=koalas_data, benchmarks=koalas_benchmarks, name='count')
benchmark(count_index_length, df=koalas_data, benchmarks=koalas_benchmarks, name='count index length')
benchmark(mean, df=koalas_data, benchmarks=koalas_benchmarks, name='mean')
benchmark(standard_deviation, df=koalas_data, benchmarks=koalas_benchmarks, name='standard deviation')
benchmark(mean_of_sum, df=koalas_data, benchmarks=koalas_benchmarks, name='mean of columns addition')
benchmark(sum_columns, df=koalas_data, benchmarks=koalas_benchmarks, name='addition of columns')
benchmark(mean_of_product, df=koalas_data, benchmarks=koalas_benchmarks, name='mean of columns multiplication')
benchmark(product_columns, df=koalas_data, benchmarks=koalas_benchmarks, name='multiplication of columns')
benchmark(value_counts, df=koalas_data, benchmarks=koalas_benchmarks, name='value counts')
benchmark(complicated_arithmetic_operation, df=koalas_data, benchmarks=koalas_benchmarks, name='complex arithmetic ops')
benchmark(mean_of_complicated_arithmetic_operation, df=koalas_data, benchmarks=koalas_benchmarks, name='mean of complex arithmetic ops')
benchmark(groupby_statistics, df=koalas_data, benchmarks=koalas_benchmarks, name='groupby statistics')
benchmark(join_count, koalas_data, benchmarks=koalas_benchmarks, name='join count', other=other)
benchmark(join_data, koalas_data, benchmarks=koalas_benchmarks, name='join', other=other)

read file took: 0.2836952209472656 seconds
count took: 0.18452048301696777 seconds
count index length took: 0.1263866424560547 seconds
mean took: 0.4323129653930664 seconds
standard deviation took: 0.564359188079834 seconds
mean of columns addition took: 0.637066125869751 seconds
addition of columns took: 0.03345060348510742 seconds
mean of columns multiplication took: 0.6672379970550537 seconds
multiplication of columns took: 0.024534225463867188 seconds
value counts took: 0.06283807754516602 seconds
complex arithmetic ops took: 0.002001523971557617 seconds
mean of complex arithmetic ops took: 0.0 seconds
groupby statistics took: 0.13896608352661133 seconds
join count took: 20.747236728668213 seconds
join took: 0.6573514938354492 seconds


#### Operations with filtering

In [13]:
expr_filter = (koalas_data.tip_amount >= 1) & (koalas_data.tip_amount <= 5)

def filter_data(df):
    return df[expr_filter]

koalas_filtered = filter_data(koalas_data)

In [14]:
benchmark(count, koalas_filtered, benchmarks=koalas_benchmarks, name='filtered count')
benchmark(count_index_length, koalas_filtered, benchmarks=koalas_benchmarks, name='filtered count index length')
benchmark(mean, koalas_filtered, benchmarks=koalas_benchmarks, name='filtered mean')
benchmark(standard_deviation, koalas_filtered, benchmarks=koalas_benchmarks, name='filtered standard deviation')
benchmark(mean_of_sum, koalas_filtered, benchmarks=koalas_benchmarks, name ='filtered mean of columns addition')
benchmark(sum_columns, df=koalas_filtered, benchmarks=koalas_benchmarks, name='filtered addition of columns')
benchmark(mean_of_product, koalas_filtered, benchmarks=koalas_benchmarks, name ='filtered mean of columns multiplication')
benchmark(product_columns, df=koalas_filtered, benchmarks=koalas_benchmarks, name='filtered multiplication of columns')
benchmark(mean_of_complicated_arithmetic_operation, koalas_filtered, benchmarks=koalas_benchmarks, name='filtered mean of complex arithmetic ops')
benchmark(complicated_arithmetic_operation, koalas_filtered, benchmarks=koalas_benchmarks, name='filtered complex arithmetic ops')
benchmark(value_counts, koalas_filtered, benchmarks=koalas_benchmarks, name ='filtered value counts')
benchmark(groupby_statistics, koalas_filtered, benchmarks=koalas_benchmarks, name='filtered groupby statistics')

other = ks.DataFrame(groupby_statistics(koalas_filtered).to_pandas())
other.columns = pd.Index([e[0]+'_' + e[1] for e in other.columns.tolist()])

benchmark(join_data, koalas_filtered, benchmarks=koalas_benchmarks, name='filtered join', other=other)
benchmark(join_count, koalas_filtered, benchmarks=koalas_benchmarks, name='filtered join count', other=other)

filtered count took: 0.7095263004302979 seconds
filtered count index length took: 0.506678581237793 seconds
filtered mean took: 0.8717646598815918 seconds
filtered standard deviation took: 1.026289701461792 seconds
filtered mean of columns addition took: 1.3728179931640625 seconds
filtered addition of columns took: 0.03542971611022949 seconds
filtered mean of columns multiplication took: 0.9695253372192383 seconds
filtered multiplication of columns took: 0.024727582931518555 seconds
filtered mean of complex arithmetic ops took: 0.0 seconds
filtered complex arithmetic ops took: 0.0 seconds
filtered value counts took: 0.05472874641418457 seconds
filtered groupby statistics took: 0.1295621395111084 seconds
filtered join took: 0.5105862617492676 seconds
filtered join count took: 21.96998405456543 seconds


#### Operations with filtering and caching

In [15]:
koalas_filtered_cached = koalas_filtered.spark.cache()
print(f'Enforce caching: {len(koalas_filtered_cached)} rows of filtered data')

Enforce caching: 4826147 rows of filtered data


In [16]:
benchmark(count, koalas_filtered, benchmarks=koalas_benchmarks, name='filtered and cached count')
benchmark(count_index_length, koalas_filtered, benchmarks=koalas_benchmarks, name='filtered and cached count index length')
benchmark(mean, koalas_filtered, benchmarks=koalas_benchmarks, name='filtered and cached mean')
benchmark(standard_deviation, koalas_filtered, benchmarks=koalas_benchmarks, name='filtered and cached standard deviation')
benchmark(mean_of_sum, koalas_filtered, benchmarks=koalas_benchmarks, name ='filtered and cached mean of columns addition')
benchmark(sum_columns, df=koalas_filtered, benchmarks=koalas_benchmarks, name='filtered and cached addition of columns')
benchmark(mean_of_product, koalas_filtered, benchmarks=koalas_benchmarks, name ='filtered and cached mean of columns multiplication')
benchmark(product_columns, df=koalas_filtered, benchmarks=koalas_benchmarks, name='filtered and cached multiplication of columns')
benchmark(mean_of_complicated_arithmetic_operation, koalas_filtered, benchmarks=koalas_benchmarks, name='filtered and cached mean of complex arithmetic ops')
benchmark(complicated_arithmetic_operation, koalas_filtered, benchmarks=koalas_benchmarks, name='filtered and cached complex arithmetic ops')
benchmark(value_counts, koalas_filtered, benchmarks=koalas_benchmarks, name ='filtered and cached value counts')
benchmark(groupby_statistics, koalas_filtered, benchmarks=koalas_benchmarks, name='filtered and cached groupby statistics')

other = ks.DataFrame(groupby_statistics(koalas_filtered).to_pandas())
other.columns = pd.Index([e[0]+'_' + e[1] for e in other.columns.tolist()])

benchmark(join_data, koalas_filtered, benchmarks=koalas_benchmarks, name='filtered and cached join', other=other)
benchmark(join_count, koalas_filtered, benchmarks=koalas_benchmarks, name='filtered and cached join count', other=other)

filtered count took: 1.5313990116119385 seconds
filtered count index length took: 1.484971046447754 seconds
filtered mean took: 1.7788598537445068 seconds
filtered standard deviation took: 1.7384052276611328 seconds
filtered mean of columns addition took: 1.8392524719238281 seconds
filtered addition of columns took: 0.03993058204650879 seconds
filtered mean of columns multiplication took: 1.8636271953582764 seconds
filtered multiplication of columns took: 0.03415489196777344 seconds
filtered mean of complex arithmetic ops took: 0.0 seconds
filtered complex arithmetic ops took: 0.0 seconds
filtered value counts took: 0.06516480445861816 seconds
filtered groupby statistics took: 0.16971755027770996 seconds
filtered join took: 0.6518526077270508 seconds
filtered join count took: 9.557752847671509 seconds


# Dask

We repeated this experiment with Dask, on the same 3 sets of operations: standard, with filtering and with filtering and caching.

In [17]:
cluster = LocalCluster(memory_limit='8GB')
client = Client(cluster)

dask_data = dd.read_parquet(paths[0])

In [18]:
dask_benchmarks = {
    'duration': [],  # in seconds
    'task': [],
}

#### Standard operations

In [19]:
def read_file_parquet(df=None):
    return dd.read_parquet("../yellow_tripdata_2011-01.parquet")

def count(df=None):
    return len(df)

def count_index_length(df=None):
    return len(df.index)

def mean(df):
    return df.fare_amount.mean().compute()

def standard_deviation(df):
    return df.fare_amount.std().compute()

def mean_of_sum(df):
    return (df.fare_amount + df.tip_amount).mean().compute()

def sum_columns(df):
    return (df.fare_amount + df.tip_amount).compute()

def mean_of_product(df):
    return (df.fare_amount * df.tip_amount).mean().compute()

def product_columns(df):
    return (df.fare_amount * df.tip_amount).compute()

def value_counts(df):
    return df.fare_amount.value_counts().compute()

#   In the original experiment, the following two functions used the longitude and latitude values of the pickup and the dropout places.
#   Since the datasets provided by NYC TLC Trip Record Data no longer have longitude and latitude values (only the pickup and dropout places IDs),
# we used arbitrary longitude and latitude values. The goal of this experiment is to compare the computational cost of the calculations, hence
# the values are not relevant.
def mean_of_complicated_arithmetic_operation(df):
    start_lon,end_lon = np.random.randint(-180,180),np.random.randint(-180,180)
    start_lat,end_lat = np.random.randint(-90,90),np.random.randint(-90,90)
    theta_1 = start_lon
    phi_1 = start_lat
    theta_2 = end_lon
    phi_2 = end_lat
    temp = (np.sin((theta_2-theta_1)/2*np.pi/180)**2
           + np.cos(theta_1*np.pi/180)*np.cos(theta_2*np.pi/180) * np.sin((phi_2-phi_1)/2*np.pi/180)**2)
    ret = 2 * np.arctan2(np.sqrt(temp), np.sqrt(1-temp))
    return ret.mean()

def complicated_arithmetic_operation(df):
    start_lon,end_lon = np.random.randint(-180,180),np.random.randint(-180,180)
    start_lat,end_lat = np.random.randint(-90,90),np.random.randint(-90,90)
    theta_1 = start_lon
    phi_1 = start_lat
    theta_2 = end_lon
    phi_2 = end_lat
    temp = (np.sin((theta_2-theta_1)/2*np.pi/180)**2
           + np.cos(theta_1*np.pi/180)*np.cos(theta_2*np.pi/180) * np.sin((phi_2-phi_1)/2*np.pi/180)**2)
    ret = 2 * np.arctan2(np.sqrt(temp), np.sqrt(1-temp))
    return ret

def groupby_statistics(df):
    return df.groupby(by='passenger_count').agg({
        'fare_amount': ['mean', 'std'],
        'tip_amount': ['mean', 'std'] 
    })

other = groupby_statistics(dask_data)
other.columns = pd.Index([e[0]+'_' + e[1] for e in other.columns.tolist()])

def join_count(df, other):
    return len(dd.merge(df, other, left_index=True, right_index=True))

def join_data(df, other):
    return dd.merge(df, other, left_index=True, right_index=True).compute()

In [20]:
benchmark(read_file_parquet, df=None, benchmarks=dask_benchmarks, name='read file')
benchmark(count, df=dask_data, benchmarks=dask_benchmarks, name='count')
benchmark(count_index_length, df=dask_data, benchmarks=dask_benchmarks, name='count index length')
benchmark(mean, df=dask_data, benchmarks=dask_benchmarks, name='mean')
benchmark(standard_deviation, df=dask_data, benchmarks=dask_benchmarks, name='standard deviation')
benchmark(mean_of_sum, df=dask_data, benchmarks=dask_benchmarks, name='mean of columns addition')
benchmark(sum_columns, df=dask_data, benchmarks=dask_benchmarks, name='addition of columns')
benchmark(mean_of_product, df=dask_data, benchmarks=dask_benchmarks, name='mean of columns multiplication')
benchmark(product_columns, df=dask_data, benchmarks=dask_benchmarks, name='multiplication of columns')
benchmark(value_counts, df=dask_data, benchmarks=dask_benchmarks, name='value counts')
benchmark(mean_of_complicated_arithmetic_operation, df=dask_data, benchmarks=dask_benchmarks, name='mean of complex arithmetic ops')
benchmark(complicated_arithmetic_operation, df=dask_data, benchmarks=dask_benchmarks, name='complex arithmetic ops')
benchmark(groupby_statistics, df=dask_data, benchmarks=dask_benchmarks, name='groupby statistics')
benchmark(join_count, dask_data, benchmarks=dask_benchmarks, name='join count', other=other)
benchmark(join_data, dask_data, benchmarks=dask_benchmarks, name='join', other=other)

read file took: 0.04585576057434082 seconds
count took: 1.3291070461273193 seconds
count index length took: 43.1181058883667 seconds
mean took: 1.2414824962615967 seconds
standard deviation took: 3.0880119800567627 seconds
mean of columns addition took: 3.27643084526062 seconds
addition of columns took: 2.332578420639038 seconds
mean of columns multiplication took: 2.07999324798584 seconds
multiplication of columns took: 2.26393985748291 seconds
value counts took: 1.158517599105835 seconds
mean of complex arithmetic ops took: 0.0 seconds
complex arithmetic ops took: 0.0 seconds
groupby statistics took: 0.06505417823791504 seconds
join count took: 30.098060846328735 seconds
join took: 18.45586347579956 seconds


#### Operations with filtering

In [21]:
expr_filter = (dask_data.tip_amount >= 1) & (dask_data.tip_amount <= 5)

def filter_data(df):
    return df[expr_filter]

dask_filtered = filter_data(dask_data)

In [22]:
benchmark(count, dask_filtered, benchmarks=dask_benchmarks, name='filtered count')
benchmark(count_index_length, dask_filtered, benchmarks=dask_benchmarks, name='filtered count index length')
benchmark(mean, dask_filtered, benchmarks=dask_benchmarks, name='filtered mean')
benchmark(standard_deviation, dask_filtered, benchmarks=dask_benchmarks, name='filtered standard deviation')
benchmark(mean_of_sum, dask_filtered, benchmarks=dask_benchmarks, name ='filtered mean of columns addition')
benchmark(sum_columns, df=dask_filtered, benchmarks=dask_benchmarks, name='filtered addition of columns')
benchmark(mean_of_product, dask_filtered, benchmarks=dask_benchmarks, name ='filtered mean of columns multiplication')
benchmark(product_columns, df=dask_filtered, benchmarks=dask_benchmarks, name='filtered multiplication of columns')
benchmark(mean_of_complicated_arithmetic_operation, dask_filtered, benchmarks=dask_benchmarks, name='filtered mean of complex arithmetic ops')
benchmark(complicated_arithmetic_operation, dask_filtered, benchmarks=dask_benchmarks, name='filtered complex arithmetic ops')
benchmark(value_counts, dask_filtered, benchmarks=dask_benchmarks, name ='filtered value counts')
benchmark(groupby_statistics, dask_filtered, benchmarks=dask_benchmarks, name='filtered groupby statistics')

other = groupby_statistics(dask_filtered)
other.columns = pd.Index([e[0] +'_'+ e[1] for e in other.columns.tolist()])

benchmark(join_count, dask_filtered, benchmarks=dask_benchmarks, name='filtered join count', other=other)
benchmark(join_data, dask_filtered, benchmarks=dask_benchmarks, name='filtered join', other=other)

filtered count took: 17.790358304977417 seconds
filtered count index length took: 16.307208776474 seconds
filtered mean took: 16.096224069595337 seconds
filtered standard deviation took: 17.178290605545044 seconds
filtered mean of columns addition took: 19.5005521774292 seconds
filtered addition of columns took: 17.45691418647766 seconds
filtered mean of columns multiplication took: 16.436156272888184 seconds
filtered multiplication of columns took: 17.223954439163208 seconds
filtered mean of complex arithmetic ops took: 0.007777690887451172 seconds
filtered complex arithmetic ops took: 0.0 seconds
filtered value counts took: 17.735454320907593 seconds
filtered groupby statistics took: 0.09854698181152344 seconds
filtered join count took: 17.671422958374023 seconds
filtered join took: 17.869524240493774 seconds


#### Operations with filtering and caching

In [23]:
dask_filtered = client.persist(dask_filtered)

from distributed import wait
print('Waiting until all futures are finished')
wait(dask_filtered)
print('All futures are finished')

Waiting until all futures are finished
All futures are finished


In [24]:
benchmark(count, dask_filtered, benchmarks=dask_benchmarks, name='filtered and cached count')
benchmark(count_index_length, dask_filtered, benchmarks=dask_benchmarks, name='filtered and cached count index length')
benchmark(mean, dask_filtered, benchmarks=dask_benchmarks, name='filtered and cached mean')
benchmark(standard_deviation, dask_filtered, benchmarks=dask_benchmarks, name='filtered and cached standard deviation')
benchmark(mean_of_sum, dask_filtered, benchmarks=dask_benchmarks, name ='filtered and cached mean of columns addition')
benchmark(sum_columns, df=dask_filtered, benchmarks=dask_benchmarks, name='filtered and cached addition of columns')
benchmark(mean_of_product, dask_filtered, benchmarks=dask_benchmarks, name ='filtered and cached mean of columns multiplication')
benchmark(product_columns, df=dask_filtered, benchmarks=dask_benchmarks, name='filtered and cached multiplication of columns')
benchmark(mean_of_complicated_arithmetic_operation, dask_filtered, benchmarks=dask_benchmarks, name='filtered and cached mean of complex arithmetic ops')
benchmark(complicated_arithmetic_operation, dask_filtered, benchmarks=dask_benchmarks, name='filtered and cached complex arithmetic ops')
benchmark(value_counts, dask_filtered, benchmarks=dask_benchmarks, name ='filtered and cached value counts')
benchmark(groupby_statistics, dask_filtered, benchmarks=dask_benchmarks, name='filtered and cached groupby statistics')

other = groupby_statistics(dask_filtered)
other.columns = pd.Index([e[0]+'_' + e[1] for e in other.columns.tolist()])

benchmark(join_count, dask_filtered, benchmarks=dask_benchmarks, name='filtered and cached join count', other=other)
benchmark(join_data, dask_filtered, benchmarks=dask_benchmarks, name='filtered and cached join', other=other)

filtered count took: 0.1582040786743164 seconds
filtered count index length took: 0.036712646484375 seconds
filtered mean took: 0.14176106452941895 seconds
filtered standard deviation took: 0.41079092025756836 seconds
filtered mean of columns addition took: 0.14326071739196777 seconds
filtered addition of columns took: 1.1182324886322021 seconds
filtered mean of columns multiplication took: 0.20514416694641113 seconds
filtered multiplication of columns took: 0.9939935207366943 seconds
filtered mean of complex arithmetic ops took: 0.0 seconds
filtered complex arithmetic ops took: 0.0 seconds
filtered value counts took: 0.14677929878234863 seconds
filtered groupby statistics took: 0.07665109634399414 seconds
filtered join count took: 1.5119473934173584 seconds
filtered join took: 0.6601705551147461 seconds


In [25]:
client.restart()

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


Client Scheduler: tcp://127.0.0.1:59597 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 29.80 GiB


## Result Analysis

In [26]:
print(koalas_benchmarks)
print(dask_benchmarks)

{'duration': [0.2836952209472656, 0.18452048301696777, 0.1263866424560547, 0.4323129653930664, 0.564359188079834, 0.637066125869751, 0.03345060348510742, 0.6672379970550537, 0.024534225463867188, 0.06283807754516602, 0.002001523971557617, 0.0, 0.13896608352661133, 20.747236728668213, 0.6573514938354492, 0.7095263004302979, 0.506678581237793, 0.8717646598815918, 1.026289701461792, 1.3728179931640625, 0.03542971611022949, 0.9695253372192383, 0.024727582931518555, 0.0, 0.0, 0.05472874641418457, 0.1295621395111084, 0.5105862617492676, 21.96998405456543, 1.5313990116119385, 1.484971046447754, 1.7788598537445068, 1.7384052276611328, 1.8392524719238281, 0.03993058204650879, 1.8636271953582764, 0.03415489196777344, 0.0, 0.0, 0.06516480445861816, 0.16971755027770996, 0.6518526077270508, 9.557752847671509], 'task': ['read file', 'count', 'count index length', 'mean', 'standard deviation', 'mean of columns addition', 'addition of columns', 'mean of columns multiplication', 'multiplication of colu

In [29]:
dask_koalas_duration_ratio = []
i=0
for task in dask_benchmarks['task']:
    dask_duration_i = dask_benchmarks['duration'][i]
    koalas_duration_i = koalas_benchmarks['duration'][i]
    if dask_duration_i == 0 and koalas_duration_i == 0:
        print(f"Task {task}: both Dask and Koalas ran this task in less time than the measurable threshold.")
        i+=1
        continue
    if dask_duration_i == 0:
        print(f"Task {task}: Dask ran this task in less time than the measurable threshold while Koalas took {koalas_duration_i} seconds.")
        i+=1
        continue
    if koalas_duration_i == 0:       
        print(f"Task {task}: Koalas ran this task in less time than the measurable threshold while Dask took {dask_duration_i} seconds.")
        i+=1
        continue
    ratio = dask_duration_i / koalas_duration_i
    dask_koalas_duration_ratio.append(ratio)
    if ratio >= 1:
        print(f"Task {task}: Koalas performs {ratio}x better than Dask")
    else:
        print(f"Task {task}: Dask performs {1/ratio}x better than Koalas")
    i+=1

Task read file: Dask performs 6.186686632039224x better than Koalas
Task count: Koalas performs 7.203032554413484x better than Dask
Task count index length: Koalas performs 341.1603081659448x better than Dask
Task mean: Koalas performs 2.8717216360439695x better than Dask
Task standard deviation: Koalas performs 5.47171383983906x better than Dask
Task mean of columns addition: Koalas performs 5.142999623135654x better than Dask
Task addition of columns: Koalas performs 69.73202805376972x better than Dask
Task mean of columns multiplication: Koalas performs 3.1173183439285155x better than Dask
Task multiplication of columns: Koalas performs 92.27680167923502x better than Dask
Task value counts: Koalas performs 18.43655382794181x better than Dask
Task mean of complex arithmetic ops: Dask ran this task in less time than the measurable threshold while Koalas took 0.002001523971557617 seconds.
Task complex arithmetic ops: both Dask and Koalas ran this task in less time than the measurable t

## Conclusions